# Membuat Dataset untuk Model R-CNN

steps:
1. convert image ke array
2. ambil nilai array dalam range boundingbox, array tersebut adalah object mobil
3. buat proposed box
4. bandingkan IoU antara boundingbox dengan proposed box, ambil nilai diatas 50% sebagai true, sisanya false
5. tulis array sebagai gambar JPG dengan folder class masing-masing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

import tensorflow as tf
import tensorflow_hub

import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img

In [2]:
# https://learnopencv.com/intersection-over-union-iou-in-object-detection-and-segmentation/
def get_iou(ground_truth, pred):
    # coordinates of the area of intersection.
    ix1 = np.maximum(ground_truth[0], pred[0])
    iy1 = np.maximum(ground_truth[1], pred[1])
    ix2 = np.minimum(ground_truth[2], pred[2])
    iy2 = np.minimum(ground_truth[3], pred[3])
    
    # Intersection height and width.
    i_height = np.maximum(iy2 - iy1 + 1, np.array(0.))
    i_width = np.maximum(ix2 - ix1 + 1, np.array(0.))
    
    area_of_intersection = i_height * i_width
    
    # Ground Truth dimensions.
    gt_height = ground_truth[3] - ground_truth[1] + 1
    gt_width = ground_truth[2] - ground_truth[0] + 1
    
    # Prediction dimensions.
    pd_height = pred[3] - pred[1] + 1
    pd_width = pred[2] - pred[0] + 1
    
    area_of_union = gt_height * gt_width + pd_height * pd_width - area_of_intersection
    
    iou = area_of_intersection / area_of_union
    
    return iou

In [3]:
img_w = 676
img_h = 380

target_w = 224
target_h = 224

In [4]:
df_boundingbox = pd.read_csv('../data/train_solution_bounding_boxes.csv')
df_boundingbox['xmin'] = df_boundingbox['xmin'].astype('int') / img_w
df_boundingbox['ymin'] = df_boundingbox['ymin'].astype('int') / img_h
df_boundingbox['xmax'] = df_boundingbox['xmax'].astype('int') / img_w
df_boundingbox['ymax'] = df_boundingbox['ymax'].astype('int') / img_h
df_boundingbox['area'] = (df_boundingbox['xmax']-df_boundingbox['xmin']) * (df_boundingbox['ymax']-df_boundingbox['ymin'])

bb_data = {}

for index,row in df_boundingbox.iterrows():
    filename, xmin, ymin, xmax, ymax = row['image'], int(row['xmin'] * target_w), int(row['ymin'] * target_h), int(row['xmax'] * target_w), int(row['ymax'] * target_h)

    if not filename in bb_data:
        bb_data[filename] = [[xmin, ymin, xmax, ymax]]
    else:
        bb_data[filename].append([xmin, ymin, xmax, ymax])

print(len(bb_data))

355


In [5]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
max_proposed_boxes = 5000

train_files = tf.io.gfile.glob('../data/training_images/*.jpg')
test_files = tf.io.gfile.glob('../data/testing_images/*.jpg')

pos_counter = 0
neg_counter = 0

for i in train_files:
    filename = i.split('\\')[-1]
    
    if filename in df_boundingbox['image'].values:
        counter = 0
        
        img = load_img(i,target_size=(target_w,target_h))
        normalized_img_array = img_to_array(img)

        ss.setBaseImage(img_to_array(img))
        ss.switchToSelectiveSearchFast()
        rects = ss.process()
        
        for (x, y, w, h) in rects:
            filename_box = filename
            proposed_box = [x, y, x + w, y + h]
            
            for x_d in bb_data.items():
                filename_bb = x_d[0]
                bounding_boxes = x_d[1]
                if filename_box == filename_bb:
                    image_array = normalized_img_array[proposed_box[1]:proposed_box[3],proposed_box[0]:proposed_box[2]]
                    image_array = cv2.resize(image_array,(target_w,target_h))

                    ious = []
                    for bounding_box in bounding_boxes:
                        ious.append(get_iou(bounding_box,proposed_box))
                    
                    if any(x > 0.5 for x in ious):
                        label = 1
                        pos_counter += 1
                        path = '../data/training_images_labeled/1/' + str(pos_counter) + '.jpg'
                        cv2.imwrite(path, image_array)
                    else:
                        label = 0
                        if (neg_counter - pos_counter) <= 25:
                            neg_counter += 1
                            path = '../data/training_images_labeled/0/' + str(neg_counter) + '.jpg'
                            cv2.imwrite(path, image_array)
                    
            counter += 1
            if counter >= max_proposed_boxes:
                break